In [1]:
import pandas as pd
import requests
import json
import sys
#put root in path
sys.path.append('../')
from src.data import helpers
from src.data import faers_translations
from copy import copy

In [2]:
faersApi = helpers.FaersApi()

# Get Data
Take top 10 interactions for each drug in list, get top 100 patient.reaction.reactionmeddrapt for them.
Take top 10 patient.reaction.reactionmeddrapt for each drug in list, get top 1000 interactions for each of them.

[best selling a-z drugs](https://www.biopharmadive.com/news/astrazeneca-pharma-dive-awards/566229/#:~:text=Lung%20cancer%20drug%20Tagrisso%20has,overtaking%20Symbicort%20earlier%20this%20year.)


In [3]:
a_z_drugs = ['symbicort','tagrisso','nexium','crestor','farxiga','brilinta','pulmicort','faslodex','zoladex','seloken','toprol-xl']
A_Z_drugs = [drug.upper() for drug in a_z_drugs]
A_Z_drugs

['SYMBICORT',
 'TAGRISSO',
 'NEXIUM',
 'CRESTOR',
 'FARXIGA',
 'BRILINTA',
 'PULMICORT',
 'FASLODEX',
 'ZOLADEX',
 'SELOKEN',
 'TOPROL-XL']

In [5]:
params_100 = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
          #, 'count':'patient.drug.medicinalproduct'
        ,'limit':100}
params_10_interacts = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
          , 'count':'patient.drug.medicinalproduct.exact'
        ,'limit':10}
params_10_reacts = {'search': {'receivedate': '[20040101+TO+20201024]'}#,'patient.drug.medicinalproduct':'symbicort'.upper()}
          , 'count':'patient.reaction.reactionmeddrapt.exact'
        ,'limit':10}

In [6]:
for drug in A_Z_drugs:
    #Get top 10 interactions
    params_10_interacts['search'].update({'patient.drug.medicinalproduct':drug})
    resp = faersApi.make_call(params_10_interacts)
    break

In [8]:
resp.json()

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2020-10-24'},
 'results': [{'term': 'SYMBICORT', 'count': 60285},
  {'term': 'SPIRIVA', 'count': 11489},
  {'term': 'SINGULAIR', 'count': 6654},
  {'term': 'PREDNISONE.', 'count': 5643},
  {'term': 'VENTOLIN', 'count': 5206},
  {'term': 'PROAIR HFA', 'count': 4295},
  {'term': 'OMEPRAZOLE.', 'count': 3709},
  {'term': 'NEXIUM', 'count': 3500},
  {'term': 'ALBUTEROL.', 'count': 3484},
  {'term': 'XOLAIR', 'count': 3307}]}